In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import  matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error

from random import sample
seed_list = list(range(10000))

import warnings
warnings.simplefilter('ignore')
os.chdir('C:\\Users\\yeonjun.in\\Desktop\\연준\\캐글\\제주도\\data')

TODAY = str(datetime.now().year)+str(datetime.now().month)+str(datetime.now().day)

In [2]:
os.chdir('C:\\Users\\yeonjun.in\\Desktop\\연준\\캐글\\제주도\\data')

train = pd.read_pickle('fixed1_train.pkl')
test = pd.read_pickle("fixed1_test.pkl")

sub = pd.read_csv('submission_sample.csv')

os.chdir('C:\\Users\\yeonjun.in\\Desktop\\연준\\캐글\\제주도\\code\\experiment')
experiment_db = pd.read_csv('experiment_DB.csv')

In [22]:
added = []
input_var=['in_out','latitude','longitude','ride_6_7', 'ride_7_8', 'ride_8_9', 
           'ride_9_10','ride_10_11', 'ride_11_12','ride_6_12',
           'takeoff_6_7', 'takeoff_7_8', 'takeoff_8_9','takeoff_9_10', 
           'takeoff_10_11', 'takeoff_11_12','takeoff_6_12',
           'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4',
           'weekday_5', 'weekday_6', 
           'dis_jejusi', 'dis_seoquipo']
target=['ride_18_20']

In [23]:
holi = ['2019-09-12','2019-09-13','2019-09-14', '2019-10-03','2019-10-09']
wkend = ['2019-09-01','2019-09-07','2019-09-08','2019-09-14','2019-09-15',
         '2019-09-21','2019-09-22','2019-09-28','2019-09-29',
        '2019-10-05','2019-10-06','2019-10-12','2019-10-13']
workday = sorted(list(set(pd.concat([train.date,test.date],axis=0).astype('str').unique()) - set(holi+wkend)))
train['day_type'] =  np.where(train.date.isin(holi),1, 
                            np.where(train.date.isin(wkend),2,3))
test['day_type'] =  np.where(test.date.isin(holi),1, 
                            np.where(test.date.isin(wkend),2,3))



train = pd.get_dummies(train,columns=['day_type'])
test = pd.get_dummies(test,columns=['day_type'])

added += [a for a in train.columns if 'day_type' in a]

In [24]:
all = pd.concat([train,test],axis=0).reset_index()

for aaa in ['bus_route_id','station_code','station_name']:
    temp = all.groupby(aaa)['id'].count().reset_index().\
    rename(columns = {'id' : str(aaa) + '_freq'})

    train = pd.merge(train,temp,how='left',on=aaa)
    test = pd.merge(test,temp,how='left',on=aaa)
    
    
    
del all
added += [a for a in train.columns if 'freq' in a]

In [25]:
os.chdir('C:\\Users\\yeonjun.in\\Desktop\\연준\\캐글\\제주도\\data')
move_18_20 = pd.read_pickle('move_18_20.pkl')
all = pd.concat([train,test],axis=0).reset_index(drop=True)

remove_outlier =\
move_18_20[~((move_18_20.x < 126.46) | ((move_18_20['x'] > 126.56) & (move_18_20['y'] > 33.5)))].reset_index(drop=True)

logic1 = ((all.latitude < 33.5) & (all.latitude > 33.47)) & ((all.longitude > 126.47) & (all.longitude < 126.50))
logic2 = ((all.latitude < 33.53) & (all.latitude > 33.48)) & ((all.longitude > 126.51) & (all.longitude < 126.54))
logic3 = ((all.latitude < 33.26) & (all.latitude > 33.24)) & ((all.longitude > 126.55) & (all.longitude < 126.57))

all['high_move'] = np.where(logic1,'1',np.where(logic2,'2',np.where(logic3,'3','0')))
all = pd.get_dummies(all,columns=['high_move'])

In [26]:
os.chdir('C:\\Users\\yeonjun.in\\Desktop\\연준\\캐글\\제주도\\data')
move_18_20 = pd.read_pickle('move_18_20.pkl')
all = pd.concat([train,test],axis=0).reset_index(drop=True)

remove_outlier =\
move_18_20[~((move_18_20.x < 126.46) | ((move_18_20['x'] > 126.56) & (move_18_20['y'] > 33.5)))].reset_index(drop=True)

logic1 = ((all.latitude < 33.5) & (all.latitude > 33.47)) & ((all.longitude > 126.47) & (all.longitude < 126.50))
logic2 = ((all.latitude < 33.53) & (all.latitude > 33.48)) & ((all.longitude > 126.51) & (all.longitude < 126.54))
logic3 = ((all.latitude < 33.26) & (all.latitude > 33.24)) & ((all.longitude > 126.55) & (all.longitude < 126.57))

all['high_move'] = np.where(logic1,'1',np.where(logic2,'2',np.where(logic3,'3','0')))
all = pd.get_dummies(all,columns=['high_move'])

train = all.loc[:(train.shape[0]-1),]
test = all.loc[train.shape[0]:,].drop('ride_18_20',axis=1).reset_index(drop=True)

del all

added += [a for a in train.columns if 'high_move' in a]

In [27]:
all = pd.concat([train,test],axis=0).reset_index(drop=True)

logic1 = ((remove_outlier.y < 33.5) & (remove_outlier.y > 33.47)) & ((remove_outlier.x > 126.47) & (remove_outlier.x < 126.50))
logic2 = ((remove_outlier.y < 33.53) & (remove_outlier.y > 33.48)) & ((remove_outlier.x > 126.51) & (remove_outlier.x < 126.54))
logic3 = ((remove_outlier.y < 33.26) & (remove_outlier.y > 33.24)) & ((remove_outlier.x > 126.55) & (remove_outlier.x < 126.57))

which1 = (remove_outlier[logic1].y.mean(), remove_outlier[logic1].x.mean())
which2 = (remove_outlier[logic2].y.mean(), remove_outlier[logic2].x.mean())
which3 = (remove_outlier[logic3].y.mean(), remove_outlier[logic3].x.mean())

import geopy.distance
all['dis_1'] = [geopy.distance.vincenty((all['latitude'].iloc[i],all['longitude'].iloc[i]), which1).km for i in range(len(all))]
all['dis_2'] = [geopy.distance.vincenty((all['latitude'].iloc[i],all['longitude'].iloc[i]), which2).km for i in range(len(all))] 
all['dis_3']  = [geopy.distance.vincenty((all['latitude'].iloc[i],all['longitude'].iloc[i]), which3).km for i in range(len(all))]

train = all.loc[:(train.shape[0]-1),]
test = all.loc[train.shape[0]:,].drop('ride_18_20',axis=1).reset_index(drop=True)

del all

added += ['dis_1','dis_2','dis_3']

In [28]:
os.chdir('C:\\Users\\yeonjun.in\\Desktop\\연준\\캐글\\제주도\\data')
sep = pd.read_pickle('sep_move.pkl')
octo = pd.read_pickle('octo_move.pkl')

total = pd.concat([sep,octo],axis=0)
total[['x','y']] = round(total[['x','y']],2)
temp = total.groupby(['x','y'])['move_18_20'].sum().reset_index().\
rename(columns = {'x' : 'longitude','y':'latitude'})

all = pd.concat([train,test],axis=0).reset_index(drop=True)
all[['latitude','longitude']] = round(all[['latitude','longitude']],2)

all = pd.merge(all,temp,how='left',on=['latitude','longitude'])
all['move_18_20'] = all['move_18_20'].fillna(0).astype('int')

train = all.loc[:(train.shape[0]-1),]
test = all.loc[train.shape[0]:,].drop('ride_18_20',axis=1).reset_index(drop=True)

del all

added += ['move_18_20']

In [29]:
all = pd.concat([train,test],axis=0).reset_index(drop=True)

for col in ['station_code','station_name','bus_route_id']:
    temp = all.groupby([col])['ride_6_12'].agg(['mean','max','min','count']).reset_index().\
        rename(columns = {'mean' : col+'_'+'ride_6_12'+'_'+'mean_morning',
                         'max' : col+'_'+'ride_6_12'+'_'+'max_morning',
                         'min' : col+'_'+'ride_6_12'+'_'+'min_morning',
                         'count' : col+'_'+'ride_6_12'+'_'+'count_morning'})
    all = pd.merge(all,temp,how='left',on=col)

train = all.loc[:(train.shape[0]-1),]
test = all.loc[train.shape[0]:,].drop('ride_18_20',axis=1).reset_index(drop=True)

del all

In [30]:
all = pd.concat([train,test],axis=0).reset_index(drop=True)

for col in ['station_code','station_name','bus_route_id']:
    temp = all.groupby([col])['takeoff_6_12'].agg(['mean','max','min','count']).reset_index().\
        rename(columns = {'mean' : col+'_'+'takeoff_6_12'+'_'+'mean_morning',
                         'max' : col+'_'+'takeoff_6_12'+'_'+'max_morning',
                         'min' : col+'_'+'takeoff_6_12'+'_'+'min_morning',
                         'count' : col+'_'+'takeoff_6_12'+'_'+'count_morning'})
    all = pd.merge(all,temp,how='left',on=col)

train = all.loc[:(train.shape[0]-1),]
test = all.loc[train.shape[0]:,].drop('ride_18_20',axis=1).reset_index(drop=True)

del all

added += [a for a in train.columns if '_morning' in a]

In [31]:
input_var = input_var + added

In [33]:
os.chdir('C:\\Users\\yeonjun.in\\Desktop\\연준\\캐글\\제주도\\data')
train.to_pickle('fixed2_train.pkl')
test.to_pickle('fixed2_test.pkl')
pd.DataFrame(columns = input_var).to_csv('fixed2_input_var.csv',index=False)